# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fbe1cd24790>
├── 'a' --> tensor([[ 1.2933,  0.8368,  1.2537],
│                   [ 2.0066, -0.4676,  0.7678]])
└── 'x' --> <FastTreeValue 0x7fbe1cd248e0>
    └── 'c' --> tensor([[-0.2849,  1.0608, -0.3752,  0.6139],
                        [ 0.5388,  0.8321, -1.2332,  1.6746],
                        [ 0.4668,  1.2991,  1.3279,  0.1124]])

In [4]:
t.a

tensor([[ 1.2933,  0.8368,  1.2537],
        [ 2.0066, -0.4676,  0.7678]])

In [5]:
%timeit t.a

65.3 ns ± 0.0179 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fbe1cd24790>
├── 'a' --> tensor([[ 1.1739,  0.0477,  1.1240],
│                   [-0.8661, -0.5175, -1.6334]])
└── 'x' --> <FastTreeValue 0x7fbe1cd248e0>
    └── 'c' --> tensor([[-0.2849,  1.0608, -0.3752,  0.6139],
                        [ 0.5388,  0.8321, -1.2332,  1.6746],
                        [ 0.4668,  1.2991,  1.3279,  0.1124]])

In [7]:
%timeit t.a = new_value

64.8 ns ± 0.0217 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.2933,  0.8368,  1.2537],
               [ 2.0066, -0.4676,  0.7678]]),
    x: Batch(
           c: tensor([[-0.2849,  1.0608, -0.3752,  0.6139],
                      [ 0.5388,  0.8321, -1.2332,  1.6746],
                      [ 0.4668,  1.2991,  1.3279,  0.1124]]),
       ),
)

In [10]:
b.a

tensor([[ 1.2933,  0.8368,  1.2537],
        [ 2.0066, -0.4676,  0.7678]])

In [11]:
%timeit b.a

55.4 ns ± 0.0263 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2174,  0.2099,  0.1864],
               [-0.8853, -0.7108, -0.4119]]),
    x: Batch(
           c: tensor([[-0.2849,  1.0608, -0.3752,  0.6139],
                      [ 0.5388,  0.8321, -1.2332,  1.6746],
                      [ 0.4668,  1.2991,  1.3279,  0.1124]]),
       ),
)

In [13]:
%timeit b.a = new_value

485 ns ± 0.0958 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

884 ns ± 21.5 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 15.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 397 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 412 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fbd50306d60>
├── 'a' --> tensor([[[ 1.2933,  0.8368,  1.2537],
│                    [ 2.0066, -0.4676,  0.7678]],
│           
│                   [[ 1.2933,  0.8368,  1.2537],
│                    [ 2.0066, -0.4676,  0.7678]],
│           
│                   [[ 1.2933,  0.8368,  1.2537],
│                    [ 2.0066, -0.4676,  0.7678]],
│           
│                   [[ 1.2933,  0.8368,  1.2537],
│                    [ 2.0066, -0.4676,  0.7678]],
│           
│                   [[ 1.2933,  0.8368,  1.2537],
│                    [ 2.0066, -0.4676,  0.7678]],
│           
│                   [[ 1.2933,  0.8368,  1.2537],
│                    [ 2.0066, -0.4676,  0.7678]],
│           
│                   [[ 1.2933,  0.8368,  1.2537],
│                    [ 2.0066, -0.4676,  0.7678]],
│           
│                   [[ 1.2933,  0.8368,  1.2537],
│                    [ 2.0066, -0.4676,  0.7678]]])
└── 'x' --> <FastTreeValue 0x7fbd50294a90>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.3 µs ± 127 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fbd50294850>
├── 'a' --> tensor([[ 1.2933,  0.8368,  1.2537],
│                   [ 2.0066, -0.4676,  0.7678],
│                   [ 1.2933,  0.8368,  1.2537],
│                   [ 2.0066, -0.4676,  0.7678],
│                   [ 1.2933,  0.8368,  1.2537],
│                   [ 2.0066, -0.4676,  0.7678],
│                   [ 1.2933,  0.8368,  1.2537],
│                   [ 2.0066, -0.4676,  0.7678],
│                   [ 1.2933,  0.8368,  1.2537],
│                   [ 2.0066, -0.4676,  0.7678],
│                   [ 1.2933,  0.8368,  1.2537],
│                   [ 2.0066, -0.4676,  0.7678],
│                   [ 1.2933,  0.8368,  1.2537],
│                   [ 2.0066, -0.4676,  0.7678],
│                   [ 1.2933,  0.8368,  1.2537],
│                   [ 2.0066, -0.4676,  0.7678]])
└── 'x' --> <FastTreeValue 0x7fbd502bbf40>
    └── 'c' --> tensor([[-0.2849,  1.0608, -0.3752,  0.6139],
                        [ 0.5388,  0.8321, -1.2332,  1.6746],
                 

In [23]:
%timeit t_cat(trees)

31 µs ± 137 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65.4 µs ± 74.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.2849,  1.0608, -0.3752,  0.6139],
                       [ 0.5388,  0.8321, -1.2332,  1.6746],
                       [ 0.4668,  1.2991,  1.3279,  0.1124]],
              
                      [[-0.2849,  1.0608, -0.3752,  0.6139],
                       [ 0.5388,  0.8321, -1.2332,  1.6746],
                       [ 0.4668,  1.2991,  1.3279,  0.1124]],
              
                      [[-0.2849,  1.0608, -0.3752,  0.6139],
                       [ 0.5388,  0.8321, -1.2332,  1.6746],
                       [ 0.4668,  1.2991,  1.3279,  0.1124]],
              
                      [[-0.2849,  1.0608, -0.3752,  0.6139],
                       [ 0.5388,  0.8321, -1.2332,  1.6746],
                       [ 0.4668,  1.2991,  1.3279,  0.1124]],
              
                      [[-0.2849,  1.0608, -0.3752,  0.6139],
                       [ 0.5388,  0.8321, -1.2332,  1.6746],
                       [ 0.4668,  1.2991,  1.3279,  0.1124]],

In [26]:
%timeit Batch.stack(batches)

79.3 µs ± 215 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.2849,  1.0608, -0.3752,  0.6139],
                      [ 0.5388,  0.8321, -1.2332,  1.6746],
                      [ 0.4668,  1.2991,  1.3279,  0.1124],
                      [-0.2849,  1.0608, -0.3752,  0.6139],
                      [ 0.5388,  0.8321, -1.2332,  1.6746],
                      [ 0.4668,  1.2991,  1.3279,  0.1124],
                      [-0.2849,  1.0608, -0.3752,  0.6139],
                      [ 0.5388,  0.8321, -1.2332,  1.6746],
                      [ 0.4668,  1.2991,  1.3279,  0.1124],
                      [-0.2849,  1.0608, -0.3752,  0.6139],
                      [ 0.5388,  0.8321, -1.2332,  1.6746],
                      [ 0.4668,  1.2991,  1.3279,  0.1124],
                      [-0.2849,  1.0608, -0.3752,  0.6139],
                      [ 0.5388,  0.8321, -1.2332,  1.6746],
                      [ 0.4668,  1.2991,  1.3279,  0.1124],
                      [-0.2849,  1.0608, -0.3752,  0.6139],
                   

In [28]:
%timeit Batch.cat(batches)

148 µs ± 236 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

317 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
